In [3]:
# making necessary imports
import os

import hdbscan
import pandas as pd
import numpy as np
import re, string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification
import torch

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim import corpora
import gensim
import nltk

# Download resources
nltk.download('vader_lexicon')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#import historical stock data from yahoo
Historical_Data_df= pd.read_csv("S&P 500 Historical Data 10 years.csv")
Historical_Data_df

,Date,Price,Open,High,Low,Vol.,Change %
0,28/06/2023,"4,376.86","4,367.48","4,390.35","4,360.22",NaN,-0.04%
1,27/06/2023,"4,378.41","4,337.36","4,384.42","4,335.00",NaN,1.15%
2,26/06/2023,"4,328.82","4,344.84","4,362.06","4,328.08",NaN,-0.45%
3,23/06/2023,"4,348.33","4,354.17","4,366.55","4,341.34",NaN,-0.77%
4,22/06/2023,"4,381.89","4,355.40","4,382.25","4,351.82",NaN,0.37%
...,...,...,...,...,...,...,...
3389,08/01/2010,"1,144.98","1,140.52","1,145.39","1,136.22",NaN,0.29%
3390,07/01/2010,"1,141.69","1,136.27","1,142.46","1,131.32",NaN,0.40%
3391,06/01/2010,"1,137.14","1,135.71","1,139.19","1,133.95",NaN,0.05%
3392,05/01/2010,"1,136.52","1,132.66","1,136.63","1,129.66",NaN,0.31%


In [5]:
#pre process the historical stock data

Historical_Data_df['Date'] = pd.to_datetime(Historical_Data_df['Date'], format='%d/%m/%Y')

# Convert 'Price' column to numeric
Historical_Data_df['Price'] = pd.to_numeric(Historical_Data_df['Price'].str.replace(',', ''), errors='coerce')

# Sort DataFrame by 'Date'
Historical_Data_df = Historical_Data_df.sort_values(by='Date')

# Calculate the difference in price compared to the previous day
Historical_Data_df['Price_Diff'] = Historical_Data_df['Price'].diff()

# Create a new column indicating 'High' or 'Low' based on the price difference
Historical_Data_df['High_Low'] = Historical_Data_df['Price_Diff'].apply(
    lambda x: 4 if x > 150 
             else (3 if x > 40 
                   else (2 if x > 0 
                         else 1))
)



In [6]:
Historical_Data_df

,Date,Price,Open,High,Low,Vol.,Change %,Price_Diff,High_Low
3393,2010-01-04,1132.99,"1,116.56","1,133.87","1,116.56",NaN,1.60%,NaN,1
3392,2010-01-05,1136.52,"1,132.66","1,136.63","1,129.66",NaN,0.31%,3.53,2
3391,2010-01-06,1137.14,"1,135.71","1,139.19","1,133.95",NaN,0.05%,0.62,2
3390,2010-01-07,1141.69,"1,136.27","1,142.46","1,131.32",NaN,0.40%,4.55,2
3389,2010-01-08,1144.98,"1,140.52","1,145.39","1,136.22",NaN,0.29%,3.29,2
...,...,...,...,...,...,...,...,...,...
4,2023-06-22,4381.89,"4,355.40","4,382.25","4,351.82",NaN,0.37%,16.20,2
3,2023-06-23,4348.33,"4,354.17","4,366.55","4,341.34",NaN,-0.77%,-33.56,1
2,2023-06-26,4328.82,"4,344.84","4,362.06","4,328.08",NaN,-0.45%,-19.51,1
1,2023-06-27,4378.41,"4,337.36","4,384.42","4,335.00",NaN,1.15%,49.59,3


In [7]:
# converting date column to datetime format
Historical_Data_df['Date'] = pd.to_datetime(Historical_Data_df['Date'],format='%d/%m/%Y')
Historical_Data_df


,Date,Price,Open,High,Low,Vol.,Change %,Price_Diff,High_Low
3393,2010-01-04,1132.99,"1,116.56","1,133.87","1,116.56",NaN,1.60%,NaN,1
3392,2010-01-05,1136.52,"1,132.66","1,136.63","1,129.66",NaN,0.31%,3.53,2
3391,2010-01-06,1137.14,"1,135.71","1,139.19","1,133.95",NaN,0.05%,0.62,2
3390,2010-01-07,1141.69,"1,136.27","1,142.46","1,131.32",NaN,0.40%,4.55,2
3389,2010-01-08,1144.98,"1,140.52","1,145.39","1,136.22",NaN,0.29%,3.29,2
...,...,...,...,...,...,...,...,...,...
4,2023-06-22,4381.89,"4,355.40","4,382.25","4,351.82",NaN,0.37%,16.20,2
3,2023-06-23,4348.33,"4,354.17","4,366.55","4,341.34",NaN,-0.77%,-33.56,1
2,2023-06-26,4328.82,"4,344.84","4,362.06","4,328.08",NaN,-0.45%,-19.51,1
1,2023-06-27,4378.41,"4,337.36","4,384.42","4,335.00",NaN,1.15%,49.59,3


In [8]:
News_Data_df= pd.read_excel("scrappednewsdata2022.xlsx")
News_Data_df

,Date,Time,Headlines
0,2022-01-01,T00:01:35+0000,The Crimson Tide will face the Georgia Bulldog...
1,2022-01-01,T00:16:10+0000,"The television star, who died on Friday at age..."
2,2022-01-01,T00:48:48+0000,"“The world looks a little different now,” said..."
3,2022-01-01,T03:00:05+0000,Peter Wentz escorts us into MMXXII with a stai...
4,2022-01-01,T04:00:35+0000,"The next challenge will come on Jan. 10, pande..."
...,...,...,...
48702,2022-12-31,T19:10:36+0000,She infused her work with political and femini...
48703,2022-12-31,T19:21:09+0000,"Heavy rain, snow and wind were battering parts..."
48704,2022-12-31,T20:18:43+0000,"Each year, the comic headlined the club during..."
48705,2022-12-31,T20:22:56+0000,"What brought the owl to the city of Cypress, i..."


In [9]:
# Removes the duplicate rows in the dataset and stores it in a new dataframe
News_Data_df=News_Data_df.drop_duplicates()

# Removes the rows with null values to avoid ambigous results
News_Data_df=News_Data_df.dropna()
News_Data_df

,Date,Time,Headlines
0,2022-01-01,T00:01:35+0000,The Crimson Tide will face the Georgia Bulldog...
1,2022-01-01,T00:16:10+0000,"The television star, who died on Friday at age..."
2,2022-01-01,T00:48:48+0000,"“The world looks a little different now,” said..."
3,2022-01-01,T03:00:05+0000,Peter Wentz escorts us into MMXXII with a stai...
4,2022-01-01,T04:00:35+0000,"The next challenge will come on Jan. 10, pande..."
...,...,...,...
48702,2022-12-31,T19:10:36+0000,She infused her work with political and femini...
48703,2022-12-31,T19:21:09+0000,"Heavy rain, snow and wind were battering parts..."
48704,2022-12-31,T20:18:43+0000,"Each year, the comic headlined the club during..."
48705,2022-12-31,T20:22:56+0000,"What brought the owl to the city of Cypress, i..."


In [10]:
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text
def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    words = sentence.split()
    filtered_sentence = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_sentence)

In [11]:
News_Data_df['text_clean'] = News_Data_df['Headlines'].apply(lambda x: filter_chars(x))
News_Data_df['text_clean'] = News_Data_df['text_clean'].apply(lambda x: remove_mult_spaces(x))
News_Data_df['text_clean'] = News_Data_df['text_clean'].apply(lambda x: clean_hashtags(x))
News_Data_df['text_clean'] = News_Data_df['text_clean'].apply(lambda x: strip_all_entities(x))
News_Data_df['text_clean'] = News_Data_df['text_clean'].apply(lambda x: remove_stopwords(x))

In [12]:
News_Data_df

,Date,Time,Headlines,text_clean
0,2022-01-01,T00:01:35+0000,The Crimson Tide will face the Georgia Bulldog...,crimson tide face georgia bulldogs jan 10 nati...
1,2022-01-01,T00:16:10+0000,"The television star, who died on Friday at age...",television star died friday age 99 shared imag...
2,2022-01-01,T00:48:48+0000,"“The world looks a little different now,” said...",world looks little different said actor ryan r...
3,2022-01-01,T03:00:05+0000,Peter Wentz escorts us into MMXXII with a stai...,peter wentz escorts us mmxxii staid sensible n...
4,2022-01-01,T04:00:35+0000,"The next challenge will come on Jan. 10, pande...",next challenge come jan 10 pandemic conditions...
...,...,...,...,...
48702,2022-12-31,T19:10:36+0000,She infused her work with political and femini...,infused work political feminist perspectives i...
48703,2022-12-31,T19:21:09+0000,"Heavy rain, snow and wind were battering parts...",heavy rain snow wind battering parts californi...
48704,2022-12-31,T20:18:43+0000,"Each year, the comic headlined the club during...",year comic headlined club holidays space closi...
48705,2022-12-31,T20:22:56+0000,"What brought the owl to the city of Cypress, i...",brought owl city cypress orange county remains...


In [13]:

unique_dates = News_Data_df['Date'].unique()

# Create a new dataframe with unique dates
unique_dates_df = pd.DataFrame({'unique_dates': unique_dates})
for i in range(1,9):
    cluster='cluster'+str(i)
    unique_dates_df[cluster]=None
    sentiment='sentiment'+str(i)
    unique_dates_df[sentiment]=None

unique_dates_df

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2022-01-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,2022-01-03,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,2022-01-04,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,2022-01-05,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
361,2022-12-28,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
362,2022-12-29,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
363,2022-12-30,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [14]:
def get_cluster_labels(ldamodel, num_words=7):
    cluster_labels = []
    topics = ldamodel.print_topics(num_words=num_words)
    for topic in topics:
        words = topic[1].split("+")
        label = " / ".join([word.split("*")[1].strip().strip('"') for word in words])
        cluster_labels.append(label)
    return cluster_labels

In [15]:
News_Data_df["Date"] = pd.to_datetime(News_Data_df["Date"])
unique_dates_df["unique_dates"] = pd.to_datetime(unique_dates_df["unique_dates"])

tokenizer = RegexpTokenizer(r'\w+')
en_stop = set(stopwords.words('english'))

In [17]:
# Create a VADER sentiment analysis object
sia = SentimentIntensityAnalyzer()


# Group by date and analyze topics and sentiment
for date, group in News_Data_df.groupby("Date"):
    sentences = group["text_clean"].tolist()

    # Skip groups with too few sentences
    if len(sentences) < 5: # you can adjust this threshold as needed
        print(f"Skipping date {date} as there are too few sentences.")
        continue
    

    texts = []
    for sentence in sentences:
        raw = sentence.lower()
        tokens = tokenizer.tokenize(raw)
        stopped_tokens = [i for i in tokens if not i in en_stop]
        texts.append(stopped_tokens)

    # Build a dictionary and corpus
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=8,             # Number of clusters
    id2word=dictionary,
    passes=15,                # More passes may lead to a more accurate model
    iterations=100,           # Increase iterations for better convergence
    chunksize=2000,           # Experiment with chunksize for efficiency
    update_every=1,
    alpha='auto',             # 'auto' learns the prior from the data
    eta='auto' )

    # Get all topics
    cluster_labels = get_cluster_labels(ldamodel)

    ClusterLabels=[]
    for idx, label in enumerate(cluster_labels):
        ClusterLabels.append(label)
    # print(ClusterLabels)

  

    # Analyze sentiment for each cluster
    cluster_sentiments = [[] for _ in range(8)]  # We have 4 clusters
    for idx, sentence in enumerate(sentences):
        bow = dictionary.doc2bow(texts[idx])
        cluster_idx = max(ldamodel[bow], key=lambda item: item[1])[0]
        sentiment_score = sia.polarity_scores(sentence)['compound']
        cluster_sentiments[cluster_idx].append(sentiment_score)

    # Print the cluster labels and their sentiment
    for idx, label in enumerate(cluster_labels):
        avg_sentiment = sum(cluster_sentiments[idx]) / len(cluster_sentiments[idx]) if cluster_sentiments[idx] else 0
       
        cohead='cluster'+str(idx+1)
        coheadsenti='sentiment'+str(idx+1)
        unique_dates_df.loc[unique_dates_df["unique_dates"] == date, cohead] = label
        unique_dates_df.loc[unique_dates_df["unique_dates"] == date, coheadsenti] = avg_sentiment
  
unique_dates_df.head()

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,country / hope / men / angry / saying / femini...,0.128222,new / people / confirmed / homes / struggling ...,-0.14002,odinet / court / foster / friday / city / inte...,0.298467,championship / georgia / jan / alabama / 10 / ...,0.3314,running / new / one / year / increasingly / mi...,0.26396,new / officer / said / different / better / da...,-0.057625,new / back / covid / dysfunction / along / com...,-0.13716,2022 / turkey / one / people / 1 / saturday / ...,-0.062667
1,2022-01-02,people / new / tribal / first / sunday / surge...,-0.005243,year / us / employees / said / every / doctor ...,0.16685,new / state / damage / york / sales / since / ...,-0.1461,many / calliste / travel / would / could / off...,-0.0566,five / company / many / stake / worth / profit...,0.2083,season / seven / row / rights / womens / sunda...,-0.0372,said / facing / korean / military / breach / r...,0.0,new / york / reader / avoid / staging / weeks ...,0.179833
2,2022-01-03,people / storm / million / democracy / without...,0.076794,2022 / start / whether / defendants / disabled...,0.057275,spoon / david / firm / political / bertolini /...,0.092178,coronavirus / pandemic / response / cases / mo...,-0.001479,day / new / city / people / pandemic / childre...,0.043489,back / year / store / cheese / long / said / c...,0.021192,people / power / changing / company / gov / ne...,0.054437,new / provided / investigation / caf / trump /...,-0.158833
3,2022-01-04,new / coronavirus / years / friend / said / 21...,-0.078519,also / many / covid19 / like / years / inspect...,-0.028922,omicron / many / years / cuomo / said / decidi...,-0.105173,said / dante / expected / three / likely / exp...,0.152035,censorship / order / major / baseball / buildi...,-0.062163,holmes / ms / coronavirus / police / tests / u...,-0.082058,two / said / new / many / one / day / group,-0.07925,four / brave / fundamentalist / fire / mark / ...,-0.222013
4,2022-01-05,one / people / conviction / late / gay / impac...,0.058122,president / new / gas / vaccinated / get / tes...,0.001906,recent / people / said / many / prominent / st...,-0.142164,say / people / public / control / process / wa...,-0.094468,said / return / jan / former / percent / early...,-0.028372,trump / variant / year / donald / house / hour...,0.118,new / led / capitol / york / jan / 6 / one,-0.155559,two / new / wave / democracies / fall / house ...,-0.018179


In [18]:
unique_dates_df

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,country / hope / men / angry / saying / femini...,0.128222,new / people / confirmed / homes / struggling ...,-0.14002,odinet / court / foster / friday / city / inte...,0.298467,championship / georgia / jan / alabama / 10 / ...,0.3314,running / new / one / year / increasingly / mi...,0.26396,new / officer / said / different / better / da...,-0.057625,new / back / covid / dysfunction / along / com...,-0.13716,2022 / turkey / one / people / 1 / saturday / ...,-0.062667
1,2022-01-02,people / new / tribal / first / sunday / surge...,-0.005243,year / us / employees / said / every / doctor ...,0.16685,new / state / damage / york / sales / since / ...,-0.1461,many / calliste / travel / would / could / off...,-0.0566,five / company / many / stake / worth / profit...,0.2083,season / seven / row / rights / womens / sunda...,-0.0372,said / facing / korean / military / breach / r...,0.0,new / york / reader / avoid / staging / weeks ...,0.179833
2,2022-01-03,people / storm / million / democracy / without...,0.076794,2022 / start / whether / defendants / disabled...,0.057275,spoon / david / firm / political / bertolini /...,0.092178,coronavirus / pandemic / response / cases / mo...,-0.001479,day / new / city / people / pandemic / childre...,0.043489,back / year / store / cheese / long / said / c...,0.021192,people / power / changing / company / gov / ne...,0.054437,new / provided / investigation / caf / trump /...,-0.158833
3,2022-01-04,new / coronavirus / years / friend / said / 21...,-0.078519,also / many / covid19 / like / years / inspect...,-0.028922,omicron / many / years / cuomo / said / decidi...,-0.105173,said / dante / expected / three / likely / exp...,0.152035,censorship / order / major / baseball / buildi...,-0.062163,holmes / ms / coronavirus / police / tests / u...,-0.082058,two / said / new / many / one / day / group,-0.07925,four / brave / fundamentalist / fire / mark / ...,-0.222013
4,2022-01-05,one / people / conviction / late / gay / impac...,0.058122,president / new / gas / vaccinated / get / tes...,0.001906,recent / people / said / many / prominent / st...,-0.142164,say / people / public / control / process / wa...,-0.094468,said / return / jan / former / percent / early...,-0.028372,trump / variant / year / donald / house / hour...,0.118,new / led / capitol / york / jan / 6 / one,-0.155559,two / new / wave / democracies / fall / house ...,-0.018179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,get / book / take / editors / stories / advant...,0.07926,little / two / hurricane / defensive / dominat...,0.22338,space / need / ukraine / get / neighbors / way...,0.130438,since / new / known / friends / 2008 / year / ...,-0.125595,took / left / also / manhattan / rooms / men / ok,-0.022067,president / tax / russian / said / many / fund...,-0.02895,new / parents / ways / health / little / mosco...,0.08268,officials / workers / civilians / rest / showi...,0.124263
361,2022-12-28,said / three / star / hours / irans / soccer /...,0.02593,make / storms / winter / states / also / know ...,0.182856,oregon / said / admitted / lying / educational...,-0.0924,right / european / critics / president / back ...,0.009667,new / would / become / painting / storm / admi...,0.136257,major / dec / wednesday / system / 28 / 2022 /...,0.034178,american / airlines / become / new / may / sai...,0.125225,many / border / another / last / 42 / said / t...,0.117562
362,2022-12-29,country / get / new / price / national / like ...,0.162919,new / said / prompted / predictions / policy /...,0.027369,new / help / people / day / officials / said /...,0.129744,plane / routine / officials / power / reporter...,0.165692,said / south / way / editor / one / robert / 2022,0.00756,said / people / us / states / military / 2023 ...,0.112177,regi

In [19]:
for i in range(1,9):
    cluster='cluster'+str(i)
    unique_dates_df[cluster] = unique_dates_df[cluster].str.split(' / ')

unique_dates_df

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,"[country, hope, men, angry, saying, feminists,...",0.128222,"[new, people, confirmed, homes, struggling, mi...",-0.14002,"[odinet, court, foster, friday, city, integrit...",0.298467,"[championship, georgia, jan, alabama, 10, nati...",0.3314,"[running, new, one, year, increasingly, mile, ...",0.26396,"[new, officer, said, different, better, day, b...",-0.057625,"[new, back, covid, dysfunction, along, company...",-0.13716,"[2022, turkey, one, people, 1, saturday, remin...",-0.062667
1,2022-01-02,"[people, new, tribal, first, sunday, surge, co...",-0.005243,"[year, us, employees, said, every, doctor, 2022]",0.16685,"[new, state, damage, york, sales, since, cases]",-0.1461,"[many, calliste, travel, would, could, officia...",-0.0566,"[five, company, many, stake, worth, profits, l...",0.2083,"[season, seven, row, rights, womens, sunday, yet]",-0.0372,"[said, facing, korean, military, breach, respo...",0.0,"[new, york, reader, avoid, staging, weeks, nin...",0.179833
2,2022-01-03,"[people, storm, million, democracy, without, f...",0.076794,"[2022, start, whether, defendants, disabled, d...",0.057275,"[spoon, david, firm, political, bertolini, suc...",0.092178,"[coronavirus, pandemic, response, cases, month...",-0.001479,"[day, new, city, people, pandemic, children, w...",0.043489,"[back, year, store, cheese, long, said, could]",0.021192,"[people, power, changing, company, gov, new, m...",0.054437,"[new, provided, investigation, caf, trump, als...",-0.158833
3,2022-01-04,"[new, coronavirus, years, friend, said, 21, next]",-0.078519,"[also, many, covid19, like, years, inspector, ...",-0.028922,"[omicron, many, years, cuomo, said, deciding, ...",-0.105173,"[said, dante, expected, three, likely, experie...",0.152035,"[censorship, order, major, baseball, building,...",-0.062163,"[holmes, ms, coronavirus, police, tests, us, s...",-0.082058,"[two, said, new, many, one, day, group]",-0.07925,"[four, brave, fundamentalist, fire, mark, sect...",-0.222013
4,2022-01-05,"[one, people, conviction, late, gay, impacted,...",0.058122,"[president, new, gas, vaccinated, get, tests, ...",0.001906,"[recent, people, said, many, prominent, states...",-0.142164,"[say, people, public, control, process, want, ...",-0.094468,"[said, return, jan, former, percent, early, home]",-0.028372,"[trump, variant, year, donald, house, hours, l...",0.118,"[new, led, capitol, york, jan, 6, one]",-0.155559,"[two, new, wave, democracies, fall, house, peo...",-0.018179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,"[get, book, take, editors, stories, advantage,...",0.07926,"[little, two, hurricane, defensive, dominating...",0.22338,"[space, need, ukraine, get, neighbors, way, yo...",0.130438,"[since, new, known, friends, 2008, year, delaw...",-0.125595,"[took, left, also, manhattan, rooms, men, ok]",-0.022067,"[president, tax, russian, said, many, fund, al...",-0.02895,"[new, parents, ways, health, little, moscow, i...",0.08268,"[officials, workers, civilians, rest, showing,...",0.124263
361,2022-12-28,"[said, three, star, hours, irans, soccer, prom...",0.02593,"[make, storms, winter, states, also, know, eff...",0.182856,"[oregon, said, admitted, lying, educational, h...",-0.0924,"[right, european, critics, president, back, am...",0.009667,"[new, would, become, painting, storm, administ...",0.136257,"[major, dec, wednesday, system, 28, 2022, fash...",0.034178,"[american, airlines, become, new, may, said, h...",0.125225,"[many, border, another, last, 42, said, title]",0.117562
362,2022-12-29,"[country, get, new, price, national, like, hel...",0.162919,"[new, said, prompted, predictions, policy, ill...",0.027369,"[new, help, people, day, officials, said, also]",0.129744,"[plane, routine, officials, power, reporter, c...",0.165692,"[said, south, way, edit

In [20]:
erererr=unique_dates_df.copy()
erererr['unique_dates'] = pd.to_datetime(erererr['unique_dates'])
erererr['date_next'] = erererr['unique_dates'] + pd.DateOffset(days=1)

In [21]:
# saving the clusters with its corresponding sentiment score in a dataframe
merged_df_final = pd.merge(erererr, Historical_Data_df, left_on='date_next', right_on='Date', how='left')
merged_df_final

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,...,date_next,Date,Price,Open,High,Low,Vol.,Change %,Price_Diff,High_Low
0,2022-01-01,"[country, hope, men, angry, saying, feminists,...",0.128222,"[new, people, confirmed, homes, struggling, mi...",-0.14002,"[odinet, court, foster, friday, city, integrit...",0.298467,"[championship, georgia, jan, alabama, 10, nati...",0.3314,"[running, new, one, year, increasingly, mile, ...",...,2022-01-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,"[people, new, tribal, first, sunday, surge, co...",-0.005243,"[year, us, employees, said, every, doctor, 2022]",0.16685,"[new, state, damage, york, sales, since, cases]",-0.1461,"[many, calliste, travel, would, could, officia...",-0.0566,"[five, company, many, stake, worth, profits, l...",...,2022-01-03,2022-01-03,4796.56,"4,778.14","4,796.64","4,758.17",NaN,0.64%,30.38,2.0
2,2022-01-03,"[people, storm, million, democracy, without, f...",0.076794,"[2022, start, whether, defendants, disabled, d...",0.057275,"[spoon, david, firm, political, bertolini, suc...",0.092178,"[coronavirus, pandemic, response, cases, month...",-0.001479,"[day, new, city, people, pandemic, children, w...",...,2022-01-04,2022-01-04,4793.54,"4,804.51","4,818.62","4,774.27",NaN,-0.06%,-3.02,1.0
3,2022-01-04,"[new, coronavirus, years, friend, said, 21, next]",-0.078519,"[also, many, covid19, like, years, inspector, ...",-0.028922,"[omicron, many, years, cuomo, said, deciding, ...",-0.105173,"[said, dante, expected, three, likely, experie...",0.152035,"[censorship, order, major, baseball, building,...",...,2022-01-05,2022-01-05,4700.58,"4,787.99","4,797.70","4,699.44",NaN,-1.94%,-92.96,1.0
4,2022-01-05,"[one, people, conviction, late, gay, impacted,...",0.058122,"[president, new, gas, vaccinated, get, tests, ...",0.001906,"[recent, people, said, many, prominent, states...",-0.142164,"[say, people, public, control, process, want, ...",-0.094468,"[said, return, jan, former, percent, early, home]",...,2022-01-06,2022-01-06,4696.05,"4,693.39","4,725.01","4,671.26",NaN,-0.10%,-4.53,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,"[get, book, take, editors, stories, advantage,...",0.07926,"[little, two, hurricane, defensive, dominating...",0.22338,"[space, need, ukraine, get, neighbors, way, yo...",0.130438,"[since, new, known, friends, 2008, year, delaw...",-0.125595,"[took, left, also, manhattan, rooms, men, ok]",...,2022-12-28,2022-12-28,3783.22,"3,829.56","3,848.32","3,780.78",NaN,-1.20%,-46.03,1.0
361,2022-12-28,"[said, three, star, hours, irans, soccer, prom...",0.02593,"[make, storms, winter, states, also, know, eff...",0.182856,"[oregon, said, admitted, lying, educational, h...",-0.0924,"[right, european, critics, president, back, am...",0.009667,"[new, would, become, painting, storm, administ...",...,2022-12-29,2022-12-29,3849.28,"3,805.45","3,858.19","3,805.45",NaN,1.75%,66.06,3.0
362,2022-12-29,"[country, get, new, price, national, like, hel...",0.162919,"[new, said, prompted, predictions, policy, ill...",0.027369,"[new, help, people, day, officials, said, also]",0.129744,"[plane, routine, officials, power, reporter, c...",0.165692,"[said, south, way, editor, one, robert, 2022]",...,2022-12-30,2022-12-30,3839.50,"3,829.06","3,839.85","3,800.34",NaN,-0.25%,-9.78,1.0
363,2022-12-30,"[years, new, teams, law, trump, one, help]",0.168669,"[year, kirill, six, months, interest, didnt, s...",0.096577,"[wider, heroes, students, four, months, book, ...",0.085,"[years, world, young, new, back, year, national]",0.105661,"[new, years, eve, mr, yakopin, 30, woman]",...,2022-12-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
file_name = 'merged_df_final_vader_1_year.xlsx'
  
# exporting the dataframe as a excel
merged_df_final.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


# Sentiment Bert

In [21]:
#making necessary imports
from transformers import pipeline
import torch


In [22]:

unique_dates1 = News_Data_df['Date'].unique()

# Create a new dataframe with unique dates
unique_dates_df1 = pd.DataFrame({'unique_dates': unique_dates})
for i in range(1,9):
    cluster='cluster'+str(i)
    unique_dates_df1[cluster]=None
    sentiment='sentiment'+str(i)
    unique_dates_df1[sentiment]=None

In [23]:
# News_Data_df["Date"] = pd.to_datetime(News_Data_df["Date"])
unique_dates_df1["unique_dates"] = pd.to_datetime(unique_dates_df1["unique_dates"])


# Load the BERT sentiment analysis pipeline
bert_sentiment_analyzer = pipeline("sentiment-analysis")


i=0
# Group by date and analyze topics and sentiment
for date, group in News_Data_df.groupby("Date"):
    print(i)
    sentences1 = group["text_clean"].tolist()

    # Skip groups with too few sentences
    if len(sentences1) < 5: # you can adjust this threshold as needed
        print(f"Skipping date {date} as there are too few sentences.")
        continue
    

    texts = []
    for sentence in sentences1:
        raw = sentence.lower()
        tokens = tokenizer.tokenize(raw)
        stopped_tokens = [i for i in tokens if not i in en_stop]
        texts.append(stopped_tokens)

    # Build a dictionary and corpus
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics=8,             # Number of clusters
    id2word=dictionary,
    passes=15,                # More passes may lead to a more accurate model
    iterations=100,           # Increase iterations for better convergence
    chunksize=2000,           # Experiment with chunksize for efficiency
    update_every=1,
    alpha='auto',             # 'auto' learns the prior from the data
    eta='auto' )

    # Get all topics
    cluster_labels1 = get_cluster_labels(ldamodel)

    ClusterLabels1=[]
    for idx, label in enumerate(cluster_labels1):
        ClusterLabels1.append(label)


    # Analyze sentiment for each cluster using BERT
    cluster_sentiments1 = [[] for _ in range(8)]  # We have 4 clusters
    for idx, sentence in enumerate(sentences1):
        bow = dictionary.doc2bow(texts[idx])
        cluster_idx1 = max(ldamodel[bow], key=lambda item: item[1])[0]

        # Calculate sentiment using BERT
        sentiment_results = bert_sentiment_analyzer(sentence)
        sentiment_score = sentiment_results[0]['score']

        cluster_sentiments1[cluster_idx1].append(sentiment_score)

    # Print the cluster labels and their sentiment
    for idx, label in enumerate(cluster_labels1):
        avg_sentiment = sum(cluster_sentiments1[idx]) / len(cluster_sentiments1[idx]) if cluster_sentiments1[idx] else 0

        cohead='cluster'+str(idx+1)
        coheadsenti='sentiment'+str(idx+1)
        unique_dates_df1.loc[unique_dates_df1["unique_dates"] == date, cohead] = label
        unique_dates_df1.loc[unique_dates_df1["unique_dates"] == date, coheadsenti] = avg_sentiment
    
    i+=1
    #breaking at 365 so the data will contain only a years data
    if i==365:
        break
unique_dates_df1

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,world / pay / friday / ryan / reynolds / tribu...,0.962016,georgia / championship / show / running / play...,0.898005,new / company / loss / one / lead / citys / mayor,0.990224,variant / day / may / banks / 75000 / collect ...,0.950761,back / people / crucial / new / 200 / years / ...,0.848328,saturday / 1 / 2022 / new / year / diet / affect,0.738002,new / people / ice / feared / fires / threats ...,0.946724,new / officer / city / promised / food / facin...,0.972592
1,2022-01-02,new / first / many / years / nfl / week / 2022,0.960452,tribal / also / need / much / find / calls / fix,0.923815,jan / calliste / new / wall / martin / product...,0.8699,sunday / surge / season / seven / row / weeks ...,0.967327,state / said / officials / pandemic / many / n...,0.984044,said / city / york / new / weeks / coronavirus...,0.990546,company / china / tear / country / crisis / na...,0.981513,year / iowa / gave / league / said / jaw / total,0.922264
2,2022-01-03,less / latest / slew / predictable / regular /...,0.969295,thousands / spoon / holmes / hundreds / winter...,0.976869,like / people / keep / know / million / nearly...,0.969647,people / coronavirus / new / omicron / pandemi...,0.94223,cheese / also / store / look / long / back / c...,0.955696,said / city / new / year / would / students / one,0.920846,caf / vaccinated / received / week / next / pa...,0.937554,mayor / gov / day / hochul / new / year / eric,0.9351
3,2022-01-04,many / 2022 / tests / police / office / cases ...,0.938022,said / three / censorship / trying / like / fo...,0.96308,new / school / charges / tuesday / aimed / deb...,0.935831,court / new / order / league / said / 21 / bas...,0.936005,two / days / day / stories / time / virus / new,0.966727,said / next / would / one / month / city / state,0.967352,trying / year / one / making / americans / pla...,0.926455,new / case / mean / life / dante / founder / s...,0.980638
4,2022-01-05,capitol / trump / house / stakes / still / don...,0.926967,j / capitol / variant / chief / told / manger ...,0.940721,people / new / first / led / trial / state / i...,0.923419,used / country / china / power / president / n...,0.916035,said / 1 / attack / week / received / lives / ...,0.929647,new / want / president / people / vaccinated /...,0.920382,new / guns / house / order / assault / said / ...,0.906407,omicron / latest / variant / burns / memory / ...,0.943052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,health / since / opposition / next / tax / rus...,0.962388,ukraine / given / shows / zimmermann / share /...,0.94241,new / book / also / known / leaders / year / f...,0.945861,many / tuesday / bucha / denial / rest / dozen...,0.928294,new / space / increasingly / supply / family /...,0.951525,washington / crossing / country / delaware / 1...,0.982403,rural / took / working / left / health / edito...,0.986635,eastern / would / look / thousands / passenger...,0.951838
361,2022-12-28,new / says / times / may / author / life / quite,0.980375,would / painting / security / president / thre...,0.973991,said / wednesday / european / oregon / would /...,0.978819,many / help / paris / popular / wilde / whose ...,0.977388,many / come / year / known / big / themes / di...,0.947285,united / states / winter / weather / know / da...,0.9139,right / american / leaving / top / one / two /...,0.916133,border / five / crossed / predecessor / north ...,0.907971
362,2022-12-29,way / says / help / service / plans / earlier ...,0.960606,business / 2023 / living / couples / speaker /...,0.959825,house / list / new / take / week / raised / co...,0.944885,new / states / times / york / united / help / ...,0.974694,said / days / people / us / get / city / back,0.949175,said / china / military / years / feet / month...,0.957233,2022 / south

In [24]:
unique_dates_df1

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,world / pay / friday / ryan / reynolds / tribu...,0.962016,georgia / championship / show / running / play...,0.898005,new / company / loss / one / lead / citys / mayor,0.990224,variant / day / may / banks / 75000 / collect ...,0.950761,back / people / crucial / new / 200 / years / ...,0.848328,saturday / 1 / 2022 / new / year / diet / affect,0.738002,new / people / ice / feared / fires / threats ...,0.946724,new / officer / city / promised / food / facin...,0.972592
1,2022-01-02,new / first / many / years / nfl / week / 2022,0.960452,tribal / also / need / much / find / calls / fix,0.923815,jan / calliste / new / wall / martin / product...,0.8699,sunday / surge / season / seven / row / weeks ...,0.967327,state / said / officials / pandemic / many / n...,0.984044,said / city / york / new / weeks / coronavirus...,0.990546,company / china / tear / country / crisis / na...,0.981513,year / iowa / gave / league / said / jaw / total,0.922264
2,2022-01-03,less / latest / slew / predictable / regular /...,0.969295,thousands / spoon / holmes / hundreds / winter...,0.976869,like / people / keep / know / million / nearly...,0.969647,people / coronavirus / new / omicron / pandemi...,0.94223,cheese / also / store / look / long / back / c...,0.955696,said / city / new / year / would / students / one,0.920846,caf / vaccinated / received / week / next / pa...,0.937554,mayor / gov / day / hochul / new / year / eric,0.9351
3,2022-01-04,many / 2022 / tests / police / office / cases ...,0.938022,said / three / censorship / trying / like / fo...,0.96308,new / school / charges / tuesday / aimed / deb...,0.935831,court / new / order / league / said / 21 / bas...,0.936005,two / days / day / stories / time / virus / new,0.966727,said / next / would / one / month / city / state,0.967352,trying / year / one / making / americans / pla...,0.926455,new / case / mean / life / dante / founder / s...,0.980638
4,2022-01-05,capitol / trump / house / stakes / still / don...,0.926967,j / capitol / variant / chief / told / manger ...,0.940721,people / new / first / led / trial / state / i...,0.923419,used / country / china / power / president / n...,0.916035,said / 1 / attack / week / received / lives / ...,0.929647,new / want / president / people / vaccinated /...,0.920382,new / guns / house / order / assault / said / ...,0.906407,omicron / latest / variant / burns / memory / ...,0.943052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,health / since / opposition / next / tax / rus...,0.962388,ukraine / given / shows / zimmermann / share /...,0.94241,new / book / also / known / leaders / year / f...,0.945861,many / tuesday / bucha / denial / rest / dozen...,0.928294,new / space / increasingly / supply / family /...,0.951525,washington / crossing / country / delaware / 1...,0.982403,rural / took / working / left / health / edito...,0.986635,eastern / would / look / thousands / passenger...,0.951838
361,2022-12-28,new / says / times / may / author / life / quite,0.980375,would / painting / security / president / thre...,0.973991,said / wednesday / european / oregon / would /...,0.978819,many / help / paris / popular / wilde / whose ...,0.977388,many / come / year / known / big / themes / di...,0.947285,united / states / winter / weather / know / da...,0.9139,right / american / leaving / top / one / two /...,0.916133,border / five / crossed / predecessor / north ...,0.907971
362,2022-12-29,way / says / help / service / plans / earlier ...,0.960606,business / 2023 / living / couples / speaker /...,0.959825,house / list / new / take / week / raised / co...,0.944885,new / states / times / york / united / help / ...,0.974694,said / days / people / us / get / city / back,0.949175,said / china / military / years / feet / month...,0.957233,2022 / south

In [25]:
for i in range(1,9):
    cluster='cluster'+str(i)
    unique_dates_df1[cluster] = unique_dates_df1[cluster].str.split(' / ')

unique_dates_df1

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,sentiment5,cluster6,sentiment6,cluster7,sentiment7,cluster8,sentiment8
0,2022-01-01,"[world, pay, friday, ryan, reynolds, tribute, ...",0.962016,"[georgia, championship, show, running, play, a...",0.898005,"[new, company, loss, one, lead, citys, mayor]",0.990224,"[variant, day, may, banks, 75000, collect, cus...",0.950761,"[back, people, crucial, new, 200, years, coach]",0.848328,"[saturday, 1, 2022, new, year, diet, affect]",0.738002,"[new, people, ice, feared, fires, threats, odi...",0.946724,"[new, officer, city, promised, food, facing, e...",0.972592
1,2022-01-02,"[new, first, many, years, nfl, week, 2022]",0.960452,"[tribal, also, need, much, find, calls, fix]",0.923815,"[jan, calliste, new, wall, martin, production,...",0.8699,"[sunday, surge, season, seven, row, weeks, var...",0.967327,"[state, said, officials, pandemic, many, new, ...",0.984044,"[said, city, york, new, weeks, coronavirus, bo...",0.990546,"[company, china, tear, country, crisis, nation...",0.981513,"[year, iowa, gave, league, said, jaw, total]",0.922264
2,2022-01-03,"[less, latest, slew, predictable, regular, int...",0.969295,"[thousands, spoon, holmes, hundreds, winter, p...",0.976869,"[like, people, keep, know, million, nearly, wa...",0.969647,"[people, coronavirus, new, omicron, pandemic, ...",0.94223,"[cheese, also, store, look, long, back, covid]",0.955696,"[said, city, new, year, would, students, one]",0.920846,"[caf, vaccinated, received, week, next, pandem...",0.937554,"[mayor, gov, day, hochul, new, year, eric]",0.9351
3,2022-01-04,"[many, 2022, tests, police, office, cases, jew...",0.938022,"[said, three, censorship, trying, like, former...",0.96308,"[new, school, charges, tuesday, aimed, debut, ...",0.935831,"[court, new, order, league, said, 21, baseball]",0.936005,"[two, days, day, stories, time, virus, new]",0.966727,"[said, next, would, one, month, city, state]",0.967352,"[trying, year, one, making, americans, plan, two]",0.926455,"[new, case, mean, life, dante, founder, silicon]",0.980638
4,2022-01-05,"[capitol, trump, house, stakes, still, donald,...",0.926967,"[j, capitol, variant, chief, told, manger, tho...",0.940721,"[people, new, first, led, trial, state, interest]",0.923419,"[used, country, china, power, president, new, ...",0.916035,"[said, 1, attack, week, received, lives, promi...",0.929647,"[new, want, president, people, vaccinated, cit...",0.920382,"[new, guns, house, order, assault, said, offer]",0.906407,"[omicron, latest, variant, burns, memory, like...",0.943052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,"[health, since, opposition, next, tax, russian...",0.962388,"[ukraine, given, shows, zimmermann, share, wee...",0.94241,"[new, book, also, known, leaders, year, fresh]",0.945861,"[many, tuesday, bucha, denial, rest, dozens, e...",0.928294,"[new, space, increasingly, supply, family, rus...",0.951525,"[washington, crossing, country, delaware, 17ye...",0.982403,"[rural, took, working, left, health, editors, ...",0.986635,"[eastern, would, look, thousands, passengers, ...",0.951838
361,2022-12-28,"[new, says, times, may, author, life, quite]",0.980375,"[would, painting, security, president, three, ...",0.973991,"[said, wednesday, european, oregon, would, day...",0.978819,"[many, help, paris, popular, wilde, whose, mor...",0.977388,"[many, come, year, known, big, themes, disappo...",0.947285,"[united, states, winter, weather, know, day, h...",0.9139,"[right, american, leaving, top, one, two, crit...",0.916133,"[border, five, crossed, predecessor, north, re...",0.907971
362,2022-12-29,"[way, says, help, service, plans, earlier, pri...",0.960606,"[business, 2023, living, couples, speaker, car...",0.959825,"[house, list, new, take, week, raised, coming]",0.944885,"[new, states, times, york, united, help, outward]",0.974694,"[said, days, people, us, get, city, back]",0.949175,"[said, c

In [26]:
erererr1=unique_dates_df1.copy()
erererr1['unique_dates'] = pd.to_datetime(erererr1['unique_dates'])
erererr1['date_next'] = erererr1['unique_dates'] + pd.DateOffset(days=1)

In [27]:
# saving the clusters with its corresponding sentiment score in a dataframe
merged_df_final1 = pd.merge(erererr1, Historical_Data_df, left_on='date_next', right_on='Date', how='left')
merged_df_final1

,unique_dates,cluster1,sentiment1,cluster2,sentiment2,cluster3,sentiment3,cluster4,sentiment4,cluster5,...,date_next,Date,Price,Open,High,Low,Vol.,Change %,Price_Diff,High_Low
0,2022-01-01,"[world, pay, friday, ryan, reynolds, tribute, ...",0.962016,"[georgia, championship, show, running, play, a...",0.898005,"[new, company, loss, one, lead, citys, mayor]",0.990224,"[variant, day, may, banks, 75000, collect, cus...",0.950761,"[back, people, crucial, new, 200, years, coach]",...,2022-01-02,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,"[new, first, many, years, nfl, week, 2022]",0.960452,"[tribal, also, need, much, find, calls, fix]",0.923815,"[jan, calliste, new, wall, martin, production,...",0.8699,"[sunday, surge, season, seven, row, weeks, var...",0.967327,"[state, said, officials, pandemic, many, new, ...",...,2022-01-03,2022-01-03,4796.56,"4,778.14","4,796.64","4,758.17",NaN,0.64%,30.38,2.0
2,2022-01-03,"[less, latest, slew, predictable, regular, int...",0.969295,"[thousands, spoon, holmes, hundreds, winter, p...",0.976869,"[like, people, keep, know, million, nearly, wa...",0.969647,"[people, coronavirus, new, omicron, pandemic, ...",0.94223,"[cheese, also, store, look, long, back, covid]",...,2022-01-04,2022-01-04,4793.54,"4,804.51","4,818.62","4,774.27",NaN,-0.06%,-3.02,1.0
3,2022-01-04,"[many, 2022, tests, police, office, cases, jew...",0.938022,"[said, three, censorship, trying, like, former...",0.96308,"[new, school, charges, tuesday, aimed, debut, ...",0.935831,"[court, new, order, league, said, 21, baseball]",0.936005,"[two, days, day, stories, time, virus, new]",...,2022-01-05,2022-01-05,4700.58,"4,787.99","4,797.70","4,699.44",NaN,-1.94%,-92.96,1.0
4,2022-01-05,"[capitol, trump, house, stakes, still, donald,...",0.926967,"[j, capitol, variant, chief, told, manger, tho...",0.940721,"[people, new, first, led, trial, state, interest]",0.923419,"[used, country, china, power, president, new, ...",0.916035,"[said, 1, attack, week, received, lives, promi...",...,2022-01-06,2022-01-06,4696.05,"4,693.39","4,725.01","4,671.26",NaN,-0.10%,-4.53,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27,"[health, since, opposition, next, tax, russian...",0.962388,"[ukraine, given, shows, zimmermann, share, wee...",0.94241,"[new, book, also, known, leaders, year, fresh]",0.945861,"[many, tuesday, bucha, denial, rest, dozens, e...",0.928294,"[new, space, increasingly, supply, family, rus...",...,2022-12-28,2022-12-28,3783.22,"3,829.56","3,848.32","3,780.78",NaN,-1.20%,-46.03,1.0
361,2022-12-28,"[new, says, times, may, author, life, quite]",0.980375,"[would, painting, security, president, three, ...",0.973991,"[said, wednesday, european, oregon, would, day...",0.978819,"[many, help, paris, popular, wilde, whose, mor...",0.977388,"[many, come, year, known, big, themes, disappo...",...,2022-12-29,2022-12-29,3849.28,"3,805.45","3,858.19","3,805.45",NaN,1.75%,66.06,3.0
362,2022-12-29,"[way, says, help, service, plans, earlier, pri...",0.960606,"[business, 2023, living, couples, speaker, car...",0.959825,"[house, list, new, take, week, raised, coming]",0.944885,"[new, states, times, york, united, help, outward]",0.974694,"[said, days, people, us, get, city, back]",...,2022-12-30,2022-12-30,3839.50,"3,829.06","3,839.85","3,800.34",NaN,-0.25%,-9.78,1.0
363,2022-12-30,"[mr, yakopin, president, hobbs, years, governm...",0.945663,"[new, years, rule, year, special, eve, legal]",0.963546,"[people, top, putin, kirill, new, 24, president]",0.872744,"[two, months, years, one, said, security, youth]",0.91008,"[new, years, began, people, pel, soccer, back]",...,2022-12-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
file_name1 = 'merged_df_final365.xlsx'
  
# expoerting the dataframe into a excel
merged_df_final1.to_excel(file_name1)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
